In [1]:
import cv2
from PIL import Image, ImageOps
import argparse
import os
import glob
import matplotlib.pyplot as plt
import multiprocessing as mp
from joblib import Parallel, delayed
import pydicom
import time
import numpy as np
import torch
import random
import pandas as pd
from pydicom.pixel_data_handlers import apply_windowing
import imageio
import time
import shutil

In [16]:
import numpy as np
x1 = np.arange(-1000, 1000)
x2 = np.arange(-1000, 1000)
mask = 2*x1 + 3*x2 + 3 == 0
x = x1[mask]
y = x2[mask]
print(list(zip(x, y)))
# x

[]


In [18]:
mask = 2*x1 + 3*x2 + 3 == 0
len(mask)

2000

# DDSM

In [2]:
ddsm = pd.read_excel("E:\WORKBASE\Project-rsna-breast-cancer-detection\MINI-DDSM-Complete-PNG-16\Data-MoreThanTwoMasks.xlsx")
ddsm = ddsm[['fullPath', 'View', 'Side', 'Status', 'Age', 'Density']]

In [3]:
ddsm['image_id'] = range(0, len(ddsm))
ddsm['image_id'] = ddsm['image_id'].apply(lambda x: f'{x}DDSM')

In [4]:
dc = {'LEFT': 'L', 'RIGHT': 'R'}
ddsm.Side = ddsm.Side.apply(lambda x: dc[x])
ddsm

,fullPath,View,Side,Status,Age,Density,image_id
0,Benign\0029\C_0029_1.LEFT_CC.png,CC,L,Benign,66.0,3,0DDSM
1,Benign\0029\C_0029_1.LEFT_MLO.png,MLO,L,Benign,66.0,3,1DDSM
2,Benign\0029\C_0029_1.RIGHT_CC.png,CC,R,Benign,66.0,3,2DDSM
3,Benign\0029\C_0029_1.RIGHT_MLO.png,MLO,R,Benign,66.0,3,3DDSM
4,Benign\0033\C_0033_1.LEFT_CC.png,CC,L,Benign,60.0,3,4DDSM
...,...,...,...,...,...,...,...
7803,Normal\4607\D_4607_1.RIGHT_MLO.png,MLO,R,Normal,41.0,1,7803DDSM
7804,Normal\4608\D_4608_1.LEFT_CC.png,CC,L,Normal,39.0,2,7804DDSM
7805,Normal\4608\D_4608_1.LEFT_MLO.png,MLO,L,Normal,39.0,2,7805DDSM
7806,Normal\4608\D_4608_1.RIGHT_CC.png,CC,R,Normal,39.0,2,7806DDSM


In [5]:
dc = {'Benign': 1, 'Normal': 0, 'Cancer': 2}
ddsm['Status'] = ddsm.Status.apply(lambda x: dc[x])

In [6]:
ddsm.head(5)

,fullPath,View,Side,Status,Age,Density,image_id
0,Benign\0029\C_0029_1.LEFT_CC.png,CC,L,1,66.0,3,0DDSM
1,Benign\0029\C_0029_1.LEFT_MLO.png,MLO,L,1,66.0,3,1DDSM
2,Benign\0029\C_0029_1.RIGHT_CC.png,CC,R,1,66.0,3,2DDSM
3,Benign\0029\C_0029_1.RIGHT_MLO.png,MLO,R,1,66.0,3,3DDSM
4,Benign\0033\C_0033_1.LEFT_CC.png,CC,L,1,60.0,3,4DDSM


In [7]:
ddsm['patient_id'] = ddsm.apply(lambda row: os.path.basename(row['fullPath']).split('_')[1], axis=1)

In [8]:
ddsm

,fullPath,View,Side,Status,Age,Density,image_id,patient_id
0,Benign\0029\C_0029_1.LEFT_CC.png,CC,L,1,66.0,3,0DDSM,0029
1,Benign\0029\C_0029_1.LEFT_MLO.png,MLO,L,1,66.0,3,1DDSM,0029
2,Benign\0029\C_0029_1.RIGHT_CC.png,CC,R,1,66.0,3,2DDSM,0029
3,Benign\0029\C_0029_1.RIGHT_MLO.png,MLO,R,1,66.0,3,3DDSM,0029
4,Benign\0033\C_0033_1.LEFT_CC.png,CC,L,1,60.0,3,4DDSM,0033
...,...,...,...,...,...,...,...,...
7803,Normal\4607\D_4607_1.RIGHT_MLO.png,MLO,R,0,41.0,1,7803DDSM,4607
7804,Normal\4608\D_4608_1.LEFT_CC.png,CC,L,0,39.0,2,7804DDSM,4608
7805,Normal\4608\D_4608_1.LEFT_MLO.png,MLO,L,0,39.0,2,7805DDSM,4608
7806,Normal\4608\D_4608_1.RIGHT_CC.png,CC,R,0,39.0,2,7806DDSM,4608


In [9]:
ddsm =ddsm.rename(columns={'View': 'view', 'Side':'laterality', 'Status':'status', 'Age':'age', 'Density':'density'})

In [10]:
ddsm

,fullPath,view,laterality,status,age,density,image_id,patient_id
0,Benign\0029\C_0029_1.LEFT_CC.png,CC,L,1,66.0,3,0DDSM,0029
1,Benign\0029\C_0029_1.LEFT_MLO.png,MLO,L,1,66.0,3,1DDSM,0029
2,Benign\0029\C_0029_1.RIGHT_CC.png,CC,R,1,66.0,3,2DDSM,0029
3,Benign\0029\C_0029_1.RIGHT_MLO.png,MLO,R,1,66.0,3,3DDSM,0029
4,Benign\0033\C_0033_1.LEFT_CC.png,CC,L,1,60.0,3,4DDSM,0033
...,...,...,...,...,...,...,...,...
7803,Normal\4607\D_4607_1.RIGHT_MLO.png,MLO,R,0,41.0,1,7803DDSM,4607
7804,Normal\4608\D_4608_1.LEFT_CC.png,CC,L,0,39.0,2,7804DDSM,4608
7805,Normal\4608\D_4608_1.LEFT_MLO.png,MLO,L,0,39.0,2,7805DDSM,4608
7806,Normal\4608\D_4608_1.RIGHT_CC.png,CC,R,0,39.0,2,7806DDSM,4608


In [11]:
ddsm = ddsm[["patient_id",	"image_id"	,"view",	"laterality"	,	"status", 'density', 'age', 'fullPath']]

In [12]:
ddsm

,patient_id,image_id,view,laterality,status,density,age,fullPath
0,0029,0DDSM,CC,L,1,3,66.0,Benign\0029\C_0029_1.LEFT_CC.png
1,0029,1DDSM,MLO,L,1,3,66.0,Benign\0029\C_0029_1.LEFT_MLO.png
2,0029,2DDSM,CC,R,1,3,66.0,Benign\0029\C_0029_1.RIGHT_CC.png
3,0029,3DDSM,MLO,R,1,3,66.0,Benign\0029\C_0029_1.RIGHT_MLO.png
4,0033,4DDSM,CC,L,1,3,60.0,Benign\0033\C_0033_1.LEFT_CC.png
...,...,...,...,...,...,...,...,...
7803,4607,7803DDSM,MLO,R,0,1,41.0,Normal\4607\D_4607_1.RIGHT_MLO.png
7804,4608,7804DDSM,CC,L,0,2,39.0,Normal\4608\D_4608_1.LEFT_CC.png
7805,4608,7805DDSM,MLO,L,0,2,39.0,Normal\4608\D_4608_1.LEFT_MLO.png
7806,4608,7806DDSM,CC,R,0,2,39.0,Normal\4608\D_4608_1.RIGHT_CC.png


In [13]:
ddsm.density.isnull().sum()

0

In [14]:
ddsm.age.isnull().sum()

12

In [15]:
ddsm[ddsm.age.isnull()]

,patient_id,image_id,view,laterality,status,density,age,fullPath
1120,1735,1120DDSM,CC,L,1,4,NaN,Benign\1735\A_1735_1.LEFT_CC.png
1121,1735,1121DDSM,MLO,L,1,4,NaN,Benign\1735\A_1735_1.LEFT_MLO.png
1122,1735,1122DDSM,CC,R,1,4,NaN,Benign\1735\A_1735_1.RIGHT_CC.png
1123,1735,1123DDSM,MLO,R,1,4,NaN,Benign\1735\A_1735_1.RIGHT_MLO.png
3948,1621,3948DDSM,CC,L,2,1,NaN,Cancer\1621\A_1621_1.LEFT_CC.png
3949,1621,3949DDSM,MLO,L,2,1,NaN,Cancer\1621\A_1621_1.LEFT_MLO.png
3950,1621,3950DDSM,CC,R,2,1,NaN,Cancer\1621\A_1621_1.RIGHT_CC.png
3951,1621,3951DDSM,MLO,R,2,1,NaN,Cancer\1621\A_1621_1.RIGHT_MLO.png
4024,1654,4024DDSM,CC,L,2,4,NaN,Cancer\1654\A_1654_1.LEFT_CC.png
4025,1654,4025DDSM,MLO,L,2,4,NaN,Cancer\1654\A_1654_1.LEFT_MLO.png


In [16]:
# Check bias

In [17]:
ddsm.age.dtype == 'float64'

True

In [18]:
import numpy as np

# Tạo dữ liệu ngẫu nhiên
def check_outlier(data, name = 'None'):
    # Tính toán Z-score cho từng giá trị dữ liệu
    # Tính toán phân vị thứ nhất và thứ ba
    q1, q3 = np.percentile(data, [25, 75])

    # Tính toán khoảng cách giữa hai phân vị này
    iqr = q3 - q1

    # Chọn ngưỡng IQR
    threshold = 2

    # Tính toán giới hạn dưới và trên của dữ liệu
    lower_bound = q1 - threshold * iqr
    upper_bound = q3 + threshold * iqr

    # Lấy các giá trị dữ liệu nằm ngoài giới hạn này làm outlier
    outliers = data[(data < lower_bound) | (data > upper_bound)]
    # In ra số lượng outlier và giá trị của chúng
    print(f'==========={name}===============')
    print(f"Số lượng outlier: {len(outliers)}")
    print("Giá trị outlier: ", outliers)
for i in ddsm.columns:
    if ddsm[i].dtype == 'float64' or ddsm[i].dtype == 'int64':
        check_outlier(ddsm[i], i)

===========status===============
Số lượng outlier: 0
Giá trị outlier:  Series([], Name: status, dtype: int64)
===========density===============
Số lượng outlier: 0
Giá trị outlier:  Series([], Name: density, dtype: int64)
===========age===============
Số lượng outlier: 0
Giá trị outlier:  Series([], Name: age, dtype: float64)


In [19]:
ddsm.age.fillna(ddsm.age.mean(), inplace=True)

C:\Users\nguye\AppData\Local\Temp\ipykernel_13260\1741459846.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ddsm.age.fillna(ddsm.age.mean(), inplace=True)


In [20]:
ddsm.age.isnull().sum()

0

Move image

In [21]:
ddsm

,patient_id,image_id,view,laterality,status,density,age,fullPath
0,0029,0DDSM,CC,L,1,3,66.0,Benign\0029\C_0029_1.LEFT_CC.png
1,0029,1DDSM,MLO,L,1,3,66.0,Benign\0029\C_0029_1.LEFT_MLO.png
2,0029,2DDSM,CC,R,1,3,66.0,Benign\0029\C_0029_1.RIGHT_CC.png
3,0029,3DDSM,MLO,R,1,3,66.0,Benign\0029\C_0029_1.RIGHT_MLO.png
4,0033,4DDSM,CC,L,1,3,60.0,Benign\0033\C_0033_1.LEFT_CC.png
...,...,...,...,...,...,...,...,...
7803,4607,7803DDSM,MLO,R,0,1,41.0,Normal\4607\D_4607_1.RIGHT_MLO.png
7804,4608,7804DDSM,CC,L,0,2,39.0,Normal\4608\D_4608_1.LEFT_CC.png
7805,4608,7805DDSM,MLO,L,0,2,39.0,Normal\4608\D_4608_1.LEFT_MLO.png
7806,4608,7806DDSM,CC,R,0,2,39.0,Normal\4608\D_4608_1.RIGHT_CC.png


In [22]:
ddsm.fullPath = ddsm.fullPath.apply(lambda x: f'E:\WORKBASE\Project-rsna-breast-cancer-detection\MINI-DDSM-Complete-PNG-16\{x}')

In [23]:
ddsm.fullPath[1]

'E:\\WORKBASE\\Project-rsna-breast-cancer-detection\\MINI-DDSM-Complete-PNG-16\\Benign\\0029\\C_0029_1.LEFT_MLO.png'

In [24]:
# os.makedirs('E:\WORKBASE\Project-rsna-breast-cancer-detection\META_DATA\Singer_view\DDSM')

In [25]:
ddsm.patient_id = ddsm.patient_id.apply(lambda x: f'{x}DDSM')

In [57]:

for i in ddsm.index:
    if os.path.exists(ddsm['fullPath'][i]) ==False:
        print(ddsm['fullPath'][i])
        print(i)
        cnt+=1

E:\WORKBASE\Project-rsna-breast-cancer-detection\MINI-DDSM-Complete-PNG-16\Cancer\0001\C_0001_1.LEFT_CC.png
2684
E:\WORKBASE\Project-rsna-breast-cancer-detection\MINI-DDSM-Complete-PNG-16\Cancer\0001\C_0001_1.LEFT_MLO.png
2685
E:\WORKBASE\Project-rsna-breast-cancer-detection\MINI-DDSM-Complete-PNG-16\Cancer\0001\C_0001_1.RIGHT_CC.png
2686
E:\WORKBASE\Project-rsna-breast-cancer-detection\MINI-DDSM-Complete-PNG-16\Cancer\0001\C_0001_1.RIGHT_MLO.png
2687
E:\WORKBASE\Project-rsna-breast-cancer-detection\MINI-DDSM-Complete-PNG-16\Cancer\0002\C_0002_1.LEFT_CC.png
2688
E:\WORKBASE\Project-rsna-breast-cancer-detection\MINI-DDSM-Complete-PNG-16\Cancer\0002\C_0002_1.LEFT_MLO.png
2689
E:\WORKBASE\Project-rsna-breast-cancer-detection\MINI-DDSM-Complete-PNG-16\Cancer\0002\C_0002_1.RIGHT_CC.png
2690
E:\WORKBASE\Project-rsna-breast-cancer-detection\MINI-DDSM-Complete-PNG-16\Cancer\0002\C_0002_1.RIGHT_MLO.png
2691
E:\WORKBASE\Project-rsna-breast-cancer-detection\MINI-DDSM-Complete-PNG-16\Cancer\0003\C

In [26]:
ddsm.loc[2684]

patient_id                                             0001DDSM
image_id                                               2684DDSM
view                                                         CC
laterality                                                    L
status                                                        2
density                                                       2
age                                                        65.0
fullPath      E:\WORKBASE\Project-rsna-breast-cancer-detecti...
Name: 2684, dtype: object

In [27]:
ddsm.to_csv('E:\WORKBASE\Project-rsna-breast-cancer-detection\META_DATA\Singer_view\DDSM\meta_ddsm.csv')

In [2]:
ddsm.status.value_counts()

NameError: name 'ddsm' is not defined

In [30]:
from sklearn.model_selection import train_test_split

In [31]:
data0 = ddsm[ddsm.status == 0].reset_index().drop('index', axis=1)
train0, test0 = train_test_split(data0, test_size=0.3, random_state=42)
test0, val0 = train_test_split(test0, test_size=0.7, random_state=42)
len(train0), len(val0), len(test0)

(1685, 507, 216)

In [32]:
data1 = ddsm[ddsm.status == 1].reset_index().drop('index', axis=1)
train1, test1 = train_test_split(data1, test_size=0.1, random_state=42)
test1, val1 = train_test_split(test1, test_size=0.7, random_state=42)
len(train1), len(val1), len(test1)

(2415, 189, 80)

In [33]:
data2 = ddsm[ddsm.status == 2].reset_index().drop('index', axis=1)
train2, test2 = train_test_split(data2, test_size=0.1, random_state=42)
test2, val2 = train_test_split(test2, test_size=0.7, random_state=42)
len(train2), len(val2), len(test2)

(2444, 191, 81)

In [34]:
train = pd.concat([train0, train1, train2]).reset_index()
test = pd.concat([test0, test1, test2]).reset_index()
val = pd.concat([val0, val1, val2]).reset_index()

In [1]:
train = pd.read_csv()

NameError: name 'train' is not defined

In [35]:
train.to_csv(r'E:\WORKBASE\Project-rsna-breast-cancer-detection\META_DATA\Singer_view\DDSM\train.csv')

In [36]:
test.to_csv(r'E:\WORKBASE\Project-rsna-breast-cancer-detection\META_DATA\Singer_view\DDSM\test.csv')

In [ ]:
val.to_csv(r'E:\WORKBASE\Project-rsna-breast-cancer-detection\META_DATA\Singer_view\DDSM\val.csv')

In [ ]:
train = pd.concat([train0, train1, train2]).reset_index()
test = pd.concat([test0, test1, test2]).reset_index()
val = pd.concat([val0, val1, val2]).reset_index()